In [1]:
import numpy as np
from tensorflow import reset_default_graph
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications.inception_v3 import InceptionV3
import cv2

C:\Users\felix\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Constants

In [2]:
IMAGE_SIZE = 150
LEARNING_RATE = 1e-3
TRAIN_PATH = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\data\\vectorized data\\train_data.npy'
TEST_PATH = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\data\\vectorized data\\test_data.npy'
IMAGE_TEST = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\nn\\test img\\img.JPG'
ONE_HOT = np.array(['American Shorthair', 'Angora', 'Ashera', 'British Shorthair',
                    'Exotic', 'Himalayan', 'Maine Coon', 'Persian', 'Ragdoll', 'Siamese', 'Sphynx'])

# Creating model

In [3]:
v3 = InceptionV3(include_top=False,
                  weights='imagenet',
                  input_shape=(150, 150, 3),
                  classes=11)

for layer in v3.layers:
    layer.trainable = False


x = GlobalAveragePooling2D(name='avg_pool')(v3.output)
x = Dense(2048, activation='tanh', name='drop_full')(x)
x = Dropout(0.5)(x)
x = Dense(11, activation='softmax', name='predictions')(x)

model = Model(v3.input, x)


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

# Load and Process Data

In [4]:
train_data, test_data = np.load(TRAIN_PATH), np.load(TEST_PATH)

train_features = train_data[:, 0]
train_features = np.array([list(i) for i in train_features]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
train_labels = train_data[:, 1]
train_labels = np.array([list(i) for i in train_labels])


test_features = test_data[:, 0]
test_features = np.array([list(i) for i in test_features]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
test_labels = test_data[:, 1]
test_labels = np.array([list(i) for i in test_labels])

# Training Model

In [5]:
# reset_default_graph()

In [6]:
tb = TensorBoard()
tb.set_model('InceptionV3')

In [7]:
model.fit(x=train_features,
          y=train_labels,
          batch_size=50,
          epochs=20,
          validation_data=(test_features, test_labels),
          verbose=1,
          callbacks=[tb],
          initial_epoch=0
         )

Train on 4514 samples, validate on 674 samples
Epoch 1/20
4514/4514 [==============================] - 17s 4ms/step - loss: 3.1106 - acc: 0.3272 - val_loss: 1.6673 - val_acc: 0.4377
Epoch 2/20
4514/4514 [==============================] - 12s 3ms/step - loss: 1.7823 - acc: 0.4364 - val_loss: 1.8956 - val_acc: 0.4525
Epoch 3/20
4514/4514 [==============================] - 12s 3ms/step - loss: 1.6428 - acc: 0.4945 - val_loss: 1.7850 - val_acc: 0.4228
Epoch 4/20
4514/4514 [==============================] - 12s 3ms/step - loss: 1.5363 - acc: 0.5239 - val_loss: 1.8120 - val_acc: 0.4332
Epoch 5/20
4514/4514 [==============================] - 12s 3ms/step - loss: 1.4324 - acc: 0.5370 - val_loss: 1.5888 - val_acc: 0.4881
Epoch 6/20
4514/4514 [==============================] - 13s 3ms/step - loss: 1.3517 - acc: 0.5636 - val_loss: 2.0033 - val_acc: 0.4184
Epoch 7/20
4514/4514 [==============================] - 13s 3ms/step - loss: 1.4092 - acc: 0.5516 - val_loss: 1.9033 - val_acc: 0.4125
Epoch 8/

In [8]:
model.save('.\\models\\InceptionV3\\I_V3-20_epochs.h5')

# Testing

In [9]:
def resize_square(image_current_path, shape):
    """Resize the image to a square shape keeping it's original proportion
    and filling the rest of image with black pads to keep it square.

    :param image_current_path: the local path where the image is.
    :type image_current_path: OS string
    :param shape: number of pixels per axis of the converted image
    :type shape: int
    :param save_path: path where the converted image must be saved
    :type save_path: OS string
    """

    original_image = cv2.imread(image_current_path, 1)
    original_size = original_image.shape[:2]  # (height, width)

    ratio = float(shape)/max(original_size)

    format_size = tuple(reversed([int(i*ratio)
                                  for i in original_size]))  # (width, height)

    resized_image = cv2.resize(original_image, format_size)

    d_w = shape - format_size[0]
    d_h = shape - format_size[1]

    top, bottom = d_h//2, d_h-(d_h//2)
    left, right = d_w//2, d_w-(d_w//2)

    color = [0, 0, 0]

    squared_image = cv2.copyMakeBorder(
        resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return squared_image

In [10]:
img = resize_square(IMAGE_TEST, IMAGE_SIZE)
img = np.array(img)/255

out = model.predict([img.reshape(-1, 150, 150, 3)])[0]

for n in range(11):
    print('{} - {:2.2f}%'.format(ONE_HOT[n], out[n]*100))

American Shorthair - 8.95%
Angora - 0.83%
Ashera - 0.00%
British Shorthair - 0.09%
Exotic - 0.01%
Himalayan - 1.87%
Maine Coon - 0.05%
Persian - 0.21%
Ragdoll - 0.73%
Siamese - 82.39%
Sphynx - 4.86%
